In [3]:
from datasets import load_dataset
dataset = load_dataset("nvidia/OpenMathInstruct-2")

print(dataset)

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

train-00000-of-00032.parquet:   0%|          | 0.00/237M [00:00<?, ?B/s]

train-00001-of-00032.parquet:   0%|          | 0.00/237M [00:00<?, ?B/s]

train-00002-of-00032.parquet:   0%|          | 0.00/237M [00:00<?, ?B/s]

...

Generating train_1M split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating train_2M split:   0%|          | 0/2000000 [00:00<?, ? examples/s]

Generating train_5M split:   0%|          | 0/5000000 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/32 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['problem', 'generated_solution', 'expected_answer', 'problem_source'],
        num_rows: 13972791
    })
    train_1M: Dataset({
        features: ['problem', 'generated_solution', 'expected_answer', 'problem_source'],
        num_rows: 1000000
    })
    train_2M: Dataset({
        features: ['problem', 'generated_solution', 'expected_answer', 'problem_source'],
        num_rows: 2000000
    })
    train_5M: Dataset({
        features: ['problem', 'generated_solution', 'expected_answer', 'problem_source'],
        num_rows: 5000000
    })
})


In [42]:
from transformers import AutoTokenizer
from dotenv import load_dotenv
from huggingface_hub import login, whoami
import os
load_dotenv()

huggingface_api_key = os.getenv("hugging_face_key")
# login(huggingface_api_key)
whoami()


{'type': 'user',
 'id': '670059fac67c24aead74fe1c',
 'name': 'vishgp',
 'fullname': 'Vishwanath Guruvayur',
 'isPro': False,
 'avatarUrl': '/avatars/59f6672f2cd4248af08204eb2ff1d68b.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'LLM_Course',
   'role': 'fineGrained',
   'createdAt': '2025-03-30T23:33:06.616Z',
   'fineGrained': {'canReadGatedRepos': True,
    'global': [],
    'scoped': [{'entity': {'_id': '670059fac67c24aead74fe1c',
       'type': 'user',
       'name': 'vishgp'},
      'permissions': ['collection.read',
       'repo.content.read',
       'repo.write',
       'collection.write']}]}}}}

In [44]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [56]:
from datasets import load_dataset
from transformers import AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\vishg\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vishg\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

C:\Users\vishg\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [57]:

def format_prompt(example):
    # Combine the problem and expected answer into a single text
    text = f"Problem: {example['problem']}\nSolution: {example['expected_answer']}"
    
    # Tokenize the combined text
    tokenized = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=512  # Adjust max_length as needed
    )
    
    # Use the same input_ids for labels
    tokenized["labels"] = tokenized["input_ids"].copy()
    
    return tokenized

# Load the dataset
train_data = dataset['train_1M']

# Tokenize the dataset
tokenized_dataset = train_data.map(
    format_prompt, 
    batched=True,
    remove_columns=train_data.column_names  # Remove original columns
)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print(tokenized_dataset[0])

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

{'input_ids': tensor(40781), 'attention_mask': tensor(1), 'labels': tensor(40781)}


In [59]:
import torch
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model


model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

lora_config = LoraConfig(
    r=8,  # Rank of the low-rank adaptation
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["c_attn", "c_proj"]  # GPT-2 specific layers
)


lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()


trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


C:\Users\vishg\AppData\Roaming\Python\Python312\site-packages\peft\tuners\lora\layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [60]:
from transformers import TrainingArguments, Trainer

# Training arguments
training_args = TrainingArguments(
    output_dir="./lora_gpt2",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_dir="./logs",
    learning_rate=2e-4,
    num_train_epochs=3,
    remove_unused_columns=True  
)

# Trainer
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset.select(range(1000)), 
)

# Start training
trainer.train()


  0%|          | 0/768000 [00:00<?, ?it/s]

C:\Users\vishg\AppData\Roaming\Python\Python312\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 7.4439, 'grad_norm': 12.416974067687988, 'learning_rate': 0.0001998697916666667, 'epoch': 0.0}


  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_runtime': 19.5862, 'eval_samples_per_second': 51.056, 'eval_steps_per_second': 25.528, 'epoch': 0.0}
{'loss': 7.268, 'grad_norm': 11.20900821685791, 'learning_rate': 0.00019973958333333334, 'epoch': 0.0}


...

  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_runtime': 12.2016, 'eval_samples_per_second': 81.956, 'eval_steps_per_second': 40.978, 'epoch': 0.09}
{'loss': 6.5895, 'grad_norm': 5.448235511779785, 'learning_rate': 0.00019401041666666669, 'epoch': 0.09}


  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_runtime': 10.4099, 'eval_samples_per_second': 96.062, 'eval_steps_per_second': 48.031, 'epoch': 0.09}


KeyboardInterrupt: 